In [1]:
from sage.algebras.flag_algebras import *

In [2]:
def _generator_graph(n):
    r"""
    Given `n` integer, generates the graphs of size `n` using nauty
    and returns them in a dictionary required for Flag constructors
    """
    for xx in graphs.nauty_geng(str(n)):
        yield {'edges': tuple(xx.edges(labels=None))}

def _identify_graph(n, ftype_points, edges):
    r"""
    Creates a unique identifier for a graph using canonical labelings
    """
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partition = list(ftype_points) + [[ii for ii in range(n) if ii not in ftype_union]]
    g = Graph([list(range(n)), edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=partition).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

TestGraphTheory = CombinatorialTheory('TestGraph', 
                                      _generator_graph, 
                                      _identify_graph, 
                                      edges=2)

In [3]:
# def _generator_threegraph(n):
#     r"""
#     Given `n` integer, generates the threegraphs of size `n` using nauty
#     and returns them in a dictionary required for Flag constructors.
    
#     Can also be modified to return hypergraphs for any size, but
#     larger edge sizes result in too large theories usually.
#     """
#     r = 3
#     for ee in range(binomial(n, r)+1):
#         for xx in hypergraphs.nauty(ee, n, uniform=r):
#             yield {'edges': xx}

# def _identify_hypergraph(n, ftype_points, edges):
#     r"""
#     Identifies hypergraphs by creating a canonical label for the adjacency
#     bipartite graph.
#     """
#     g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
#               format='vertices_and_edges')
#     ftype_union = [jj for ff in ftype_points for jj in ff]
#     partt = list(ftype_points) + \
#             [[ii for ii in range(n) if ii not in ftype_union]] + \
#             [list(range(n,n+len(edges)))]
#     blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
#     return (n, tuple([len(xx) for xx in ftype_points]), blocks)


# TestThreeGraphTheory = CombinatorialTheory('TestThreeGraph', 
#                                       _generator_threegraph, 
#                                       _identify_hypergraph, 
#                                       edges=3)

# G = TestThreeGraphTheory
# G.clear()
# G.exclude(G(5))
# res = optimize_problem(G, G(4), 6, certificate=True)

In [69]:
res = optimize_problem(TGT, TGT(4), 6, maximize=True, y_optimal=optim)

base flags generated
avals constructed
ftypes constructed
typed flags constructed, their length is [12, 63, 64, 64, 64, 64]
sym and asym matrices constructed
done with mult table for Ftype on 2 points with edges=[]
done with mult table for Ftype on 4 points with edges=[]
done with mult table for Ftype on 4 points with edges=[[0, 1, 2]]
done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]]
done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]]
done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]]
tables constructed
tables added to sdp data
constraints constructed
constraints added to sdp data
Running SDP after kernel correction with block sizes [3, 5, 9, 50, 17, 46, 24, 40, 18, 46, 10, 53, -2102, -2]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 2.59e-01 Pobj: -2.4276410e+01 Ad: 2.99e-01 Dobj:  2.3001812e+01 
Iter:  2 Ap: 6.77e-01 P

In [65]:
def blowup_construction(self, target_size, construction_size, **kwargs):
    from tqdm import tqdm
    res = -1
    for verts in tqdm(itertools.product(range(construction_size), repeat=int(target_size))):
        #print(verts)
        blocks = {}
        for rel in kwargs:
            if rel not in self.signature():
                continue
            reledges = kwargs[rel]
            bladd = []
            for edge in reledges:
                clusters = [[ii for ii in range(target_size) if verts[ii]==ee] for ee in edge]
                bladd += [xx for xx in itertools.product(*clusters) if len(set(xx))==len(edge) and xx[0]==min(xx) and xx[-1]==max(xx)]
            blocks[rel] = bladd
        #print(self(target_size, **blocks))
        res += self(target_size, **blocks)
    return (res+1)/(construction_size**target_size)

In [4]:
def sym_asym_bases(self, n, ftype=None):
    r"""
    Generate the change of base matrices for the symmetric
    and the asymmetric subspaces
    """
    
    flags = self.generate_flags(n, ftype)
    uniques = []
    sym_base = []
    asym_base = []
    for xx in flags:
        xxid = self.identify(n, [xx.ftype()], **xx.blocks())
        if xxid not in uniques:
            uniques.append(xxid)
            sym_base.append(xx.afae())
        else:
            sym_ind = uniques.index(xxid)
            asym_base.append(sym_base[sym_ind] - xx.afae())
            sym_base[sym_ind] += xx
    m_sym = matrix(len(sym_base), len(flags), [xx.values() for xx in sym_base], sparse=True)
    m_asym = matrix(len(asym_base), len(flags), [xx.values() for xx in asym_base], sparse=True)
    return m_sym, m_asym

In [5]:
#rounding based on ldl decomposition and continued fractions

def print_sparse(ls, eps=1e-6):
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#continued fraction rounding
def cfr(value, quotient_bound=7, denom_bound=9):
    cf = continued_fraction(value)
    for ii, xx in enumerate(cf.quotients()):
        if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
            if ii>0:
                return cf.convergent(ii-1)
            return 0
    return cf.value()

#rounding for lists
def cfr_list(ls, force_pos=False):
    if force_pos:
        return [max(cfr(xx), 0) for xx in ls]
    else:
        return [cfr(xx) for xx in ls]

#rounding for matrices
def cfr_matrix(mat):
    return matrix(QQ, [cfr_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def cfr_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, cfr_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

def sdem(value, den=1024):
    return floor(value*den)/den

#rounding for lists
def sdem_list(ls, force_pos=False):
    if force_pos:
        return [max(sdem(xx), 0) for xx in ls]
    else:
        return [sdem(xx) for xx in ls]

#rounding for matrices
def sdem_matrix(mat):
    return matrix(QQ, [sdem_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def sdem_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, sdem_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, sdem_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

def cfr_adaptive(ls, onevec):
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([cfr(xx, resol1, resol2) for xx in ls])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    return best_vec

In [68]:
#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#continued fraction rounding
def cfr(value, quotient_bound=7, denom_bound=9):
    cf = continued_fraction(value)
    for ii, xx in enumerate(cf.quotients()):
        if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
            if ii>0:
                return cf.convergent(ii-1)
            return 0
    return cf.value()

#rounding for lists
def cfr_list(ls, force_pos=False):
    if force_pos:
        return [max(cfr(xx), 0) for xx in ls]
    else:
        return [cfr(xx) for xx in ls]

#rounding for matrices
def cfr_matrix(mat):
    return matrix(QQ, [cfr_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def cfr_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, cfr_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#rounding based on simple denominator scaling
def sdem(value, den=1024):
    return floor(value*den)/den

#rounding for lists
def sdem_list(ls, force_pos=False):
    if force_pos:
        return [max(sdem(xx), 0) for xx in ls]
    else:
        return [sdem(xx) for xx in ls]

#rounding for matrices
def sdem_matrix(mat):
    return matrix(QQ, [sdem_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def sdem_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, sdem_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, sdem_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#adaptive continued fraction rounding, seems to work well
def cfr_adaptive(ls, onevec):
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([cfr(xx, resol1, resol2) for xx in ls])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    return best_vec


#helper function, to add the kernel constraints to the table constructor
def adjust_table_constructor(self, table_constructor, y_vector):
    for param in table_constructor.keys():
        ns, ftype = param
        table = self.mul_project_table(ns, ns, ftype, [])
        
        Zs = [None]*len(table_constructor[param])
        for gg, morig in enumerate(table):
            for ii, base in enumerate(table_constructor[param]):
                mat = base * morig * base.T
                if Zs[ii]==None:
                    Zs[ii] = mat*y_vector[gg]
                else:
                    Zs[ii] += mat*y_vector[gg]
        
        new_bases = []
        for ii, Z in enumerate(Zs):
            if min(Z.eigenvalues())<0:
                print("Kernel for type {} is not semidef: {}".format(ftype, min(Z.eigenvalues())))
            Zkern = Z.kernel().basis_matrix()
            if Zkern.nrows()>0:
                new_bases.append(Zkern * table_constructor[param][ii])
        table_constructor[param] = new_bases

    return table_constructor

#helper function, moves a table to an sdp input data
def tables_to_sdp_data(self, table_constructor, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        block_sizes += [base.nrows() for base in table_constructor[params]]
        
        #only loop through the table once
        for gg, morig in enumerate(table):
            #for each base change create the entries
            for plus_index, base in enumerate(table_constructor[params]):
                mm = base * morig * base.T
                dd = mm._dict()
                if len(dd)>0:
                    inds, values = zip(*mm._dict().items())
                    iinds, jinds = zip(*inds)
                    for cc in range(len(iinds)):
                        if iinds[cc]>=jinds[cc]:
                            mat_inds.extend([gg+1, block_index + plus_index, iinds[cc]+1, jinds[cc]+1])
                            mat_vals.append(values[cc])
        block_index += len(table_constructor[params])
    return mat_inds, mat_vals, block_sizes

#helper function, moves the linear constraints to the sdp input data
def constraints_to_sdp_data(self, flag_num, constraints_vals, constraints_flags_vec, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1

    constr_num = len(constraints_vals)
    
    for ii in range(constr_num):
        mat_inds.extend([0, block_index+1, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    for gg in range(flag_num):
        mat_inds.extend([gg+1, block_index, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(constr_num):
            mat_inds.extend([gg+1, block_index+1, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    block_sizes += [-flag_num, -constr_num]
    
    return mat_inds, mat_vals, block_sizes

#flatten and unflatten symmetric matrices
def flatten_matrix(mat, doubled=False):
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def unflatten_matrix(ls, dim, doubled=False):
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

#round the sdp output based on slacks
def round_sdp_solution(self, sdp_result, table_constructor, avals, y_rounded, block_sizes, constraints_flags_vec):
    linear_vals_num = -block_sizes[-1]
    linear_vals = sdp_result['X'][-1]
    linear_slacks_inds = [ii for ii in range(linear_vals_num) if abs(linear_vals[ii])>1e-8]
    linear_slacks_rounded = sdem_list([linear_vals[ii] for ii in linear_slacks_inds])
    
    
    no_slack_inds = [ii for ii in range(len(y_rounded)) if y_rounded[ii]!=0]
    target_value = avals*y_rounded

    linear_coeffs = matrix(QQ, len(linear_slacks_inds), len(no_slack_inds), \
                           [[constraints_flags_vec[ii][jj] for jj in no_slack_inds] for ii in linear_slacks_inds])
    
    #
    # Flatten the matrices (A for the no_slack graphs and X)
    #
    
    Aflat = linear_coeffs.T
    Xflat = linear_slacks_rounded
    block_index = 0
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            nls = sdem_list(flatten_matrix(sdp_result['X'][block_index + plus_index]))
            Xflat += sdem_list(flatten_matrix(sdp_result['X'][block_index + plus_index]))
            
            Aadded = []
            
            for sharp in no_slack_inds:
                morig = table[sharp]
                Aadded.append(flatten_matrix((base * morig * base.T).rows(), doubled=True))
            
            Aflat = Aflat.augment(matrix(Aadded))
        block_index += len(table_constructor[params])

    #
    # Correct the X values, so they are tight on graphs without slack
    #
    
    Xflat = vector(Xflat)
    tight_slacks = vector([avals[ii] - target_value for ii in no_slack_inds])
    Xflat_corr = Xflat - Aflat.T*(Aflat*Aflat.T).pseudoinverse()*(Aflat*Xflat - tight_slacks)
    
    #
    # Recover the X matrices from the corrected flat X
    #

    

    linear_slacks_corrected = Xflat_corr[:len(linear_slacks_inds)]
    linear_vals_corrected = vector(QQ, linear_vals_num, dict(zip(linear_slacks_inds, Xflat_corr)))
    if min(linear_slacks_corrected)<0:
        print("Linear coefficient is negative: {}".format(min(linear_slacks_corrected)))
    X_final = []
    
    Xflat_corr = Xflat_corr[len(linear_slacks_inds):]
    for dd in block_sizes:
        if dd<0:
            break
        Xdd, Xflat_corr = unflatten_matrix(Xflat_corr, dd)
        X_final.append(Xdd)
        if min(Xdd.eigenvalues())<0:
            print("Rounded X matrix is not semidefinite: {}".format(min(Xdd.eigenvalues())))
    X_final.append(linear_vals_corrected)
    linear_coeffs_all = matrix(QQ, constraints_flags_vec)
    
    #
    # Verify the bound and semidefiniteness
    #

    block_index = 0
    slacks = vector(avals) - linear_coeffs_all.T*linear_vals_corrected
    
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            Xflat = vector(flatten_matrix(X_final[block_index + plus_index].rows()))
            
            for gg, morig in enumerate(table):
                mm = base * morig * base.T
                matflat = flatten_matrix((base * morig * base.T).rows(), doubled=True)
                slacks[gg] -= vector(matflat)*Xflat
        
        block_index += len(table_constructor[params])
    
    return min(slacks), X_final

#the combined code
def optimize_problem(self, target_element, target_size, maximize=True, positives=None, y_optimal=None, certificate=False):
    from csdpy import solve_sdp
    
    #
    # initial setup
    #

    base_flags = self.generate_flags(target_size)
    print("base flags generated")
    mult = -1 if maximize else 1
    avals = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    print("avals constructed")
    
    #
    # create the table data
    #

    ftype_flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
              for flag in self.generate_flags(kk)]
    ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
              for flag in ftype_flags]
    print("ftypes constructed")
    flags = [self.generate_flags((target_size + ftype.size())//2, ftype) for ftype in ftypes]
    flag_sizes = [len(xx) for xx in flags]
    print("typed flags constructed, their length is {}".format(flag_sizes))
    
    sym_asym_mats = [sym_asym_bases(self, (target_size + ftype.size())//2, ftype) for ftype in ftypes]
    print("sym and asym matrices constructed")
    
    table_constructor = {}
    for ii, ftype in enumerate(ftypes):
        ns = (target_size + ftype.size())//2
        #pre-calculate the table here
        table = self.mul_project_table(ns, ns, ftype, [])
        sym_base, asym_base = sym_asym_mats[ii]
        bases = []
        if sym_base.nrows()!=0:
            bases.append(sym_base)
        if asym_base.nrows()!=0:
            bases.append(asym_base)
        table_constructor[(ns, ftype)] = bases
        print("done with mult table for {}".format(ftype))
    print("tables constructed")
    
    sdp_data = tables_to_sdp_data(self, table_constructor)
    print("tables added to sdp data")
    
    #
    # add constraints data
    #
    
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
            print("done with constraint for \n{}\n".format(fv))
        constraints_vals = [0]*len(constraints_flags)
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    constraints_flags_vec = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    print("constraints constructed")
    
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, constraints_flags_vec, sdp_data)
    print("constraints added to sdp data")
    
    #
    # if no y value provided, run the optimizer first, only to get the y values
    #
    if y_optimal==None or len(y_optimal)!=len(base_flags) or min(y_optimal.values())<0 or sum(y_optimal.values())!=1:
        mat_inds, mat_vals, block_sizes = sdp_data
        initial_sol = solve_sdp(block_sizes, list(avals), mat_inds, mat_vals)
        global y_original
        global one_vec
        y_original = initial_sol['y']
        one_vec = one_vector.values()
        y_rounded = cfr_adaptive(initial_sol['y'], one_vector.values())
        print("rounded y vector is: \n{}".format(print_sparse(y_rounded)))
    else:
        y_rounded = y_optimal.values()
    
    #
    # adjust the table to consider the kernel from y_rounded
    #
    
    table_constructor = adjust_table_constructor(self, table_constructor, y_rounded)
    sdp_data = tables_to_sdp_data(self, table_constructor)
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, constraints_flags_vec, sdp_data)
    mat_inds, mat_vals, block_sizes = sdp_data
    
    print("Running SDP after kernel correction with block sizes {}".format(block_sizes))
    final_sdp = solve_sdp(block_sizes, list(avals), mat_inds, mat_vals)
    
    res = round_sdp_solution(self, final_sdp, table_constructor, avals, y_rounded, block_sizes, constraints_flags_vec)
    if maximize:
        res = (-res[0], res[1])
    if certificate:
        return res
    else:
        return res[0]

In [24]:
G = TestGraphTheory
G.clear()
G.exclude(G(3))
max_deg = 1/3 - G(2, ftype=[0])
res = optimize_problem(G, G(2), 3, positives=[max_deg], certificate=True)
res

base flags generated
avals constructed
ftypes constructed
typed flags constructed, their length is [2]
sym and asym matrices constructed
done with mult table for Ftype on 1 points with edges=[]
tables constructed
tables added to sdp data
done with constraint for 
Flag Algebra Element over Rational Field
-2/3 - Flag on 2 points, ftype from [0] with edges=[]
1/3  - Flag on 2 points, ftype from [0] with edges=[[0, 1]]

constraints constructed
constraints added to sdp data
rounded y vector is: 
0: 1/3
1: 1/3
2: 1/3
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -1.5864839e+01 Ad: 7.71e-01 Dobj:  5.2455714e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -1.4302694e+01 Ad: 9.38e-01 Dobj: -1.4482965e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -4.7787650e+00 Ad: 8.72e-01 Dobj: -1.7773193e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.0441293e+00 Ad: 8.26e-01 Dobj: -1.8815758e-01 
Iter:  5 Ap: 9.75e-01 Pobj: -4.1263857e-01 Ad: 8.72e-01 Dobj: -2.3591544e-01 
Iter

(1/3, [[2427/3904], (15089/7808, 8335/15616, 579/512, 579/512)])

In [7]:
def parse_flagmatic_flag(st):
    n = int(st[0])
    eds = st[2:]
    if st[-1]==")":
        k = int(st[-2])
        eds = st[2:-3]
    edges = []
    curr = None
    for xx in eds:
        if curr==None:
            curr = int(xx)
        else:
            edges.append([curr-1, int(xx)-1])
            curr = None
    return TestGraphTheory(n, edges=edges, ftype=list(range(k)))

arr_flags_str = [
        [
            "3:(1)", 
            "3:12(1)", 
            "3:23(1)", 
            "3:1223(1)", 
            "3:1213(1)", 
            "3:121323(1)"
        ], 
        [
            "4:(3)", 
            "4:14(3)", 
            "4:24(3)", 
            "4:34(3)", 
            "4:1424(3)", 
            "4:1434(3)", 
            "4:2434(3)", 
            "4:142434(3)"
        ], 
        [
            "4:12(3)", 
            "4:1214(3)", 
            "4:1224(3)", 
            "4:1234(3)", 
            "4:121424(3)", 
            "4:121434(3)", 
            "4:122434(3)", 
            "4:12142434(3)"
        ], 
        [
            "4:1213(3)", 
            "4:121314(3)", 
            "4:121324(3)", 
            "4:121334(3)", 
            "4:12131424(3)", 
            "4:12131434(3)", 
            "4:12132434(3)", 
            "4:1213142434(3)"
        ], 
        [
            "4:121323(3)", 
            "4:12131423(3)", 
            "4:12132324(3)", 
            "4:12132334(3)", 
            "4:1213142324(3)", 
            "4:1213142334(3)", 
            "4:1213232434(3)", 
            "4:121314232434(3)"
        ]
    ]

flags_other = [[parse_flagmatic_flag(xx) for xx in flags_str] for flags_str in arr_flags_str]
flags_other

[[Flag on 3 points, ftype from [0] with edges=[],
  Flag on 3 points, ftype from [0] with edges=[[0, 1]],
  Flag on 3 points, ftype from [0] with edges=[[1, 2]],
  Flag on 3 points, ftype from [0] with edges=[[0, 1], [1, 2]],
  Flag on 3 points, ftype from [0] with edges=[[0, 1], [0, 2]],
  Flag on 3 points, ftype from [0] with edges=[[0, 1], [0, 2], [1, 2]]],
 [Flag on 4 points, ftype from [0, 1, 2] with edges=[],
  Flag on 4 points, ftype from [0, 1, 2] with edges=[[0, 3]],
  Flag on 4 points, ftype from [0, 1, 2] with edges=[[1, 3]],
  Flag on 4 points, ftype from [0, 1, 2] with edges=[[2, 3]],
  Flag on 4 points, ftype from [0, 1, 2] with edges=[[0, 3], [1, 3]],
  Flag on 4 points, ftype from [0, 1, 2] with edges=[[0, 3], [2, 3]],
  Flag on 4 points, ftype from [0, 1, 2] with edges=[[1, 3], [2, 3]],
  Flag on 4 points, ftype from [0, 1, 2] with edges=[[0, 3], [1, 3], [2, 3]]],
 [Flag on 4 points, ftype from [0, 1, 2] with edges=[[0, 1]],
  Flag on 4 points, ftype from [0, 1, 2] wit

In [89]:
def optimize_problem(self, target_element, target_size, maximize=True, \
                     ftypes=None, positives=None, \
                     certificate=False, rounding=None):
    import time
    from csdpy import solve_sdp
    from tqdm import tqdm
    import sys
    global debugmat
    global block_sizes
    
    current = time.time()
    #calculate constraints from positive vectors
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    #calculate ftypes
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]

    print("Ftypes constructed in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    block_sizes = [len(self.generate_flags((target_size + \
                   ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])

    block_sizes.extend([-constraints, -len(constraints_vals)])
    block_num = len(block_sizes)
    mat_inds = []
    mat_vals = []
    print("Block sizes done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    print("Block sizes are {}".format(block_sizes), flush=True)
    print("Calculating product matrices for {} ftypes and {} structures".format(len(ftypes), constraints), flush=True)
    for ii, ftype in (pbar := tqdm(enumerate(ftypes), file=sys.stdout)):
        ns = (target_size + ftype.size())//2
        fls = self.generate_flags(ns, ftype)
        table = self.mul_project_table(ns, ns, ftype, [])
        for gg, mm in enumerate(table):
            dd = mm._dict()
            if len(dd)>0:
                inds, values = zip(*mm._dict().items())
                iinds, jinds = zip(*inds)
                for cc in range(len(iinds)):
                    if iinds[cc]>=jinds[cc]:
                        mat_inds.extend([gg+1, ii+1, iinds[cc]+1, 
                                         jinds[cc]+1])
                        mat_vals.append(values[cc])
        pbar.set_description("{} is complete".format(ftype))
    
    print("Table calculation done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    if maximize:
        avals = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        avals = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    for ii in range(len(constraints_vals)):
        mat_inds.extend([0, block_num, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    constraints_flags_vec = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    for gg in range(constraints):
        mat_inds.extend([gg+1, block_num-1, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(len(constraints_flags_vec)):
            mat_inds.extend([gg+1, block_num, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    print("Target and constraint calculation done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()

    debugmat = matrix(QQ, len(mat_inds)/4, mat_inds)
    debugmat = debugmat.augment(matrix([mat_vals]).T)
    
    sdp_result = solve_sdp(block_sizes, list(avals), mat_inds, mat_vals)
    if maximize:
        ret = max(-sdp_result['primal'], -sdp_result['dual'])
    else:
        ret = min(sdp_result['primal'], sdp_result['dual'])
    print("Result is {}".format(ret), flush=True)
    if certificate:
        ralg = FlagAlgebra(RR, self)
        vec = ralg(target_size, sdp_result['y'])
        ret = (ret, vec, sdp_result)
    return ret

In [ ]:
def optimize_problem(self, target_element, target_size, maximize=True, \
                     ftypes=None, positives=None, \
                     certificate=False, rounding=None):
    import time
    import sys

    try:
        from csdpy import solve_sdp
    except:
        print("No csdpy solver, can't run the optimizer!")
        return
    
    current = time.time()
    #calculate constraints from positive vectors
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    #calculate ftypes
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]
    
    print("Ftypes constructed in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    flag_sizes = [len(self.generate_flags((target_size + ftype.size())//2, ftype)) for ftype in ftypes]
    sym_asym_mats = [sym_asym_bases(self, (target_size + ftype.size())//2, ftype) for ftype in ftypes]
    block_sizes = [xx.nrows() for sasx in sym_asym_mats for xx in sasx]
    constraints = len(self.generate_flags(target_size))
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    
    block_sizes.extend([-constraints, -len(constraints_vals)])
    block_num = len(block_sizes)
    next_block_index = 1
    mat_inds = []
    mat_vals = []
    print("Block sizes done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    print("Block sizes are {}".format(block_sizes), flush=True)
    print("Calculating product matrices for {} ftypes and {} structures".format(len(ftypes), constraints), flush=True)
    
    pbar = None
    has_tqdm = True
    try:
        from tqdm import tqdm
        pbar = tqdm(enumerate(ftypes), file=sys.stdout)
    except:
        pass
    pbar = enumerate(ftypes)
    has_tqdm = False
    
    for ii, ftype in pbar:
        ns = (target_size + ftype.size())//2
        fls = self.generate_flags(ns, ftype)
        table = self.mul_project_table(ns, ns, ftype, [])
        sym_base, asym_base = sym_asym_mats[ii]
        for gg, morig in enumerate(table):
            if block_sizes[2*ii]!=0:
                mm = sym_base*morig*sym_base.T
                dd = mm._dict()
                if len(dd)>0:
                    inds, values = zip(*mm._dict().items())
                    iinds, jinds = zip(*inds)
                    for cc in range(len(iinds)):
                        if iinds[cc]>=jinds[cc]:
                            mat_inds.extend([gg+1, next_block_index, iinds[cc]+1, 
                                             jinds[cc]+1])
                            mat_vals.append(values[cc])
            if block_sizes[2*ii + 1]!=0:
                mm = asym_base*morig*asym_base.T
                dd = mm._dict()
                if len(dd)>0:
                    inds, values = zip(*mm._dict().items())
                    iinds, jinds = zip(*inds)
                    for cc in range(len(iinds)):
                        if iinds[cc]>=jinds[cc]:
                            mat_inds.extend([gg+1, next_block_index + 1, iinds[cc]+1, 
                                             jinds[cc]+1])
                            mat_vals.append(values[cc])
        if block_sizes[2*ii]!=0:
            next_block_index += 1
        if block_sizes[2*ii+1]!=0:
            next_block_index += 1
        if has_tqdm: 
            pbar.set_description("{} is complete".format(ftype))
        else:
            print("{} is complete".format(ftype), flush=True)
    
    print("Table calculation done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    if maximize:
        avals = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        avals = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    for ii in range(len(constraints_vals)):
        mat_inds.extend([0, next_block_index+1, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    constraints_flags_vec = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    for gg in range(constraints):
        mat_inds.extend([gg+1, next_block_index, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(len(constraints_flags_vec)):
            mat_inds.extend([gg+1, next_block_index+1, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    print("Target and constraint calculation done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()

    block_sizes_nonzero = [bs for bs in block_sizes if bs != 0]

    combined = matrix(QQ, len(mat_inds)/4, mat_inds)
    combined = combined.augment(matrix([mat_vals]).T)

    #print("\n\ncombined data is:\n{}\n\n".format(combined))
    
    sdp_result = solve_sdp(block_sizes_nonzero, list(avals), 
                           mat_inds, mat_vals)
    print("SDP finished in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()

    # check scipy is installed, so we can do LP for the rounding
    if rounding != None:
        try:
            from scipy.optimize import linprog
        except:
            print("SciPy is needed for the rounding!")
            rounding=None
    
    if rounding==None:
        #No rounding, just return the result
        if maximize:
            ret = -sdp_result['primal']
        else:
            ret = sdp_result['primal']
        print("Result is {}".format(ret), flush=True)
        if certificate:
            ralg = FlagAlgebra(RR, self)
            vec = ralg(target_size, sdp_result['y'])
            ret = (ret, vec, sdp_result)
        return ret
    else:
        #
        # Rounding code
        #
        
        accuracy = 10**max(rounding, 2)
        
        #First get a good rounded y:
        print("Rounding dual", flush=True); current = time.time()
        yvec = vector(sdp_result['y'])
        onevec = one_vector.values()
        
        best_y = None
        best_err = 1000
        
        for ex in range(1, 8):
            ry = vector(cfr_list(yvec, 10**ex))
            prod = onevec*ry
            
            if prod != 0 and abs(prod-1)<best_err:
                best_err = abs(prod-1)
                best_y = ry/prod
        
        #Then loop through the tables, keeping track the y-corrected and
        #normal slack values

        slack_base = vector(list(avals))
        slack_corr = vector(list(avals))
        
        correct_y = True
        
        pbar = None
        if has_tqdm:
            pbar = tqdm(enumerate(ftypes), file=sys.stdout)
        else:
            pbar = enumerate(ftypes)

        next_block_index = 0
        
        for ii, ftype in pbar:
            #calculation of the table entries
            ns = (target_size + ftype.size())//2
            fls = self.generate_flags(ns, ftype)
            table = self.mul_project_table(ns, ns, ftype, [])
            sym_base, asym_base = sym_asym_mats[ii]
            
            #if still correct, calculate the Z matrix
            if correct_y:
                Zii_sym = None
                Zii_asym = None
                for jj, morig in enumerate(table):
                    if block_sizes[2*ii]!=0:
                        mat = sym_base*morig*sym_base.T
                        if Zii_sym==None:
                            Zii_sym = mat*best_y[jj]
                        else:
                            Zii_sym += mat*best_y[jj]
                    if block_sizes[2*ii+1]!=0:
                        mat = asym_base*morig*asym_base.T
                        if Zii_asym==None:
                            Zii_asym = mat*best_y[jj]
                        else:
                            Zii_asym += mat*best_y[jj]
                if block_sizes[2*ii]!=0 and min(Zii_sym.eigenvalues())<0:
                    correct_y = False
                if correct_y and block_sizes[2*ii + 1]!=0 and min(Zii_asym.eigenvalues())<0:
                    correct_y = False
            
            #the two different X matrix rounding
            if Zii_sym != None:
                Xii_base_sym = cfr_ldl(sdp_result['X'][next_block_index], accuracy)
                Xii_base_sym = vector(Xii_base_sym.list())
                if correct_y:
                    Xii_corr_sym = cfr_ldl_corr(sdp_result['X'][next_block_index], Zii_sym, accuracy)
                    Xii_corr_sym = vector(Xii_corr_sym.list())
                next_block_index += 1
            else:
                Xii_base_sym = vector([])
                Xii_corr_sym = vector([])
            if Zii_asym != None:
                Xii_base_asym = cfr_ldl(sdp_result['X'][next_block_index], accuracy)
                Xii_base_asym = vector(Xii_base_asym.list())
                if correct_y:
                    Xii_corr_asym = cfr_ldl_corr(sdp_result['X'][next_block_index], Zii_asym, accuracy)
                    Xii_corr_asym = vector(Xii_corr_asym.list())
                next_block_index += 1
            else:
                Xii_base_asym = vector([])
                Xii_corr_asym = vector([])
            
            #update slacks
            for jj, morig in enumerate(table):
                #TODO: make this a sparse vector, as the table is sparse
                mat_sym = sym_base*morig*sym_base.T
                mat_asym = asym_base*morig*asym_base.T
                mvec_sym = vector(mat_sym.list())
                mvec_asym = vector(mat_asym.list())
                base_prod = (Xii_base_sym*mvec_sym) + (Xii_base_asym*mvec_asym)
                slack_base[jj] -= base_prod
                if correct_y:
                    corr_prod = (Xii_corr_sym*mvec_sym) + (Xii_corr_asym*mvec_asym)
                    slack_corr[jj] -= corr_prod
            if has_tqdm:
                pbar.set_description("{} is complete".format(ftype))
            else:
                print("{} is complete".format(ftype), flush=True)
        
        #
        # semidefinite effect on slacks is done
        # optimize the rest with LP
        # 
        #
        #
        # LP has form:
        #
        # linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, 
        #         bounds=(0, None), method='highs', callback=None, 
        #         options=None, x0=None, integrality=None)
        #
        # minimize_x c * x
        # s.t. A_ub * x <= b_ub
        #      A_eq * x == b_eq
        #      bound[0] <= x <= bounds[1]
        # 
        # want to use method='highs-ds' for simplex
        
        sdim = len(constraints_flags_vec)-2
        
        A_ub = matrix(QQ, [-onevec] + constraints_flags_vec[:sdim]).T
        b_ub = list(slack_corr) if correct_y else list(slack_base)
        bounds = [[None] + [0]*sdim, [None]*(1 + sdim)]
        c = [1] + [0]*sdim
        lpsol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs-ds")
        
        print("Dual rounding done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()
        
        lpvals = vector(cfr_list(lpsol.x[1:], True))
        A_ubs = matrix(QQ, sdim, constraints, constraints_flags_vec[:sdim]).T
        final_slack = vector(b_ub) - A_ubs*lpvals
        slack_quotient = [-final_slack[ii]/onevec[ii] for ii in range(len(onevec)) if onevec[ii]!=0]
        return max(slack_quotient)